In [1]:
import src.decorrelate as cluster_model
MODEL_NAME = 'EleutherAI/pythia-70m'
DATASET_NAME = 'NeelNanda/pile-10k'

N_DIMS = 512
SEED = 69_420
 
DEBUG = False
 
if DEBUG:
    N_DATASIZE = 300
    N_BLOCKS = 12
    STRING_SIZE_CUTOFF = 200
else:
    # It gets killed aroun 1_800 idk why. Maybe we have a problem with token truncation somewhere
    N_DATASIZE = 1_800
# 
    # N_CLUSTERS_MIN = int(0.5 * N_DIMS)
    # N_CLUSTERS_MAX = 10 * N_DIMS
    # TODO: DEL ME
    N_BLOCKS = 6
    STRING_SIZE_CUTOFF = 1_200

params = cluster_model.InterpParams(
	lattice_params=cluster_model.LatticeParams(
		top_layer_idx = -1,
        max_n_parents = 4
	),
    # quantization='4bit',
	seed=SEED,
    n_datasize=N_DATASIZE,
    n_blocks=N_BLOCKS,
    model_name=MODEL_NAME,
	model_n_dims=N_DIMS,
    dataset_name=DATASET_NAME,
    string_size_cutoff=STRING_SIZE_CUTOFF,
    quantization='4bit'
)

decomp = cluster_model.Decomposer(params)
decomp.load()

/home/lev/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Creating decomposer with parameter data hash cache/data-2d2fdedfff35e3dd9c277c71e0d9221ab34f1d69/start.pkl
Creating decomposer with parameter lattice hash cache/correlation-8a54ca0a541e764ad00f25c2bf884c2cac448157/start.pkl
Created dataset
Got embeddings
Loading dataset from cache
Using saved correlation for layer 0
Using saved correlation for layer 1
Using saved correlation for layer 2
Using saved correlation for layer 3
Using saved correlation for layer 4
Using saved correlation for layer 5


## Lets just look at the highest cliques

In [2]:
decomp.internal_correlations[1]

array([[1.        , 0.        , 0.49433812, ..., 0.3761467 , 0.43882282,
        0.54184486],
       [0.        , 1.        , 0.48773998, ..., 0.38884602, 0.40527423,
        0.57321223],
       [0.46509317, 0.51539929, 1.        , ..., 0.37196671, 0.43882171,
        0.54031173],
       ...,
       [0.45388721, 0.52699675, 0.47706647, ..., 1.        , 0.41510652,
        0.56630892],
       [0.48119008, 0.49913273, 0.5114459 , ..., 0.37722141, 1.        ,
        0.        ],
       [0.44769753, 0.53194262, 0.47450225, ..., 0.38776844, 0.        ,
        1.        ]])

## Build Up a "Concept" Lattice Using Graph Restrictions

> TODO: this is for latter...

In [3]:
import src.graph as graph
import networkx as nx

LAYER = 3

G = graph.graph_from_correlations(decomp.internal_correlations[LAYER])
MAX_CLIQUE_SIZE = 1_000

GG = graph.sparsify_weighted_graph(G, degree_upperbound=400)
i = nx.find_cliques(GG)
clique = next(i)

In [4]:
import importlib
import src.utils as utils
importlib.reload(utils)

# decomp.tune_clique((0.0, clique), 3)

<module 'src.utils' from '/home/lev/code/research/ai/lattice_cluster_paths/src/utils.py'>

In [5]:
# import networkx as nx

# test_at = 100
# GG = graph.sparsify_weighted_graph(G, degree_upperbound=test_at)

# cliques = []

# for node in range(N_DIMS * 2):
#     if node % 50 == 0:
#         print("On node", node)
#     c_it = nx.find_cliques(GG, nodes=[node])
#     c_in_curr = []
#     for c in c_it:
#         c_in_curr.append(c)
#         if len(c_in_curr) > 50:
#             break
#     cliques += c_in_curr
#         # if len(cliques) > 10_000:
#         #     break

## Score a cell

In [6]:
decomp.internal_correlations[0]

array([[1.        , 0.        , 0.39302223, ..., 0.41651665, 0.40421225,
        0.4464732 ],
       [0.        , 1.        , 0.42954468, ..., 0.44745615, 0.33309616,
        0.58418726],
       [0.37108543, 0.47733753, 1.        , ..., 0.38395448, 0.36606731,
        0.53831451],
       ...,
       [0.33878978, 0.42835995, 0.33076601, ..., 1.        , 0.28211922,
        0.52123939],
       [0.45105037, 0.43746735, 0.43263309, ..., 0.38703506, 1.        ,
        0.        ],
       [0.27322067, 0.42075636, 0.34889717, ..., 0.39215473, 0.        ,
        1.        ]])

In [9]:
decomp.scores_for_neuron(3, 0, degree_upperbound=200)

TypeError: Decomposer.scores_for_neuron() got an unexpected keyword argument 'degree_upperbound'